In [1]:
import pyemu
import os, shutil
import pandas as pd
import flopy
import numpy as np
import platform

# This notebook is designed to show high-level PST file generation and manipulation examples, and also highlight some of the steps that `Schur` object functions perform in a bit more detail

The assumption is that a user has a model that `flopy` can grok. That's all we need, using the BOSS PEST-ification techniques `pyemu` has. 

In [2]:
nam_file = "freyberg.nam"
org_model_ws = "Freyberg_transient"
new_model_ws = "pest_setup"
EXE_DIR = 'bin'
if os.path.exists(EXE_DIR):
    if "window" in platform.platform().lower():
        exe_files = [f for f in os.listdir(EXE_DIR) if f.endswith('exe')]
    else:
        exe_files = [f for f in os.listdir(EXE_DIR) if not f.endswith('exe')]
    [shutil.copy2(os.path.join(EXE_DIR,f),os.path.join('temp',f)) for f in exe_files]
# load the model, change dir and run once to get a hydmod output file and list file
m = flopy.modflow.Modflow.load(nam_file,model_ws=org_model_ws,check=False)
m.change_model_ws("temp",reset_external=True)
m.name = nam_file.split(".")[0]

# let's just retain the calibration data for now by trimming HYDMOD
hyd = m.get_package('hyd')
hyddf = pd.DataFrame(hyd.obsdata)
hyddf = hyddf.loc[[True if 'cr' in i.decode() else False for i in hyddf.hydlbl]]
hyd.obsdata = hyddf.to_records(index=False).astype(hyd.obsdata.dtype)
hyd.nhyd = len(hyd.obsdata)
m.exe_name = 'mfnwt'
m.write_input()
pyemu.helpers.run('{0} {1}'.format(m.exe_name,nam_file), cwd='temp')


changing model workspace...
   temp
run():mfnwt freyberg.nam


## Let's make some parameters. How about zones for HK and a constants for SY and RCH?

In [3]:
zn_array = np.loadtxt(os.path.join("Freyberg_truth","hk.zones"))
k_zone_dict = {k:zn_array for k in range(m.nlay)}
const_props = []
const_props.append(["upw.sy", None])
const_props.append(["rch.rech",None])
zone_props = [['upw.hk',0]]

## Maybe we ought to also treat well pumping as an uncertain parameter

In [4]:
m.model_ws

'temp'

In [5]:
bc_props = [["wel.flux",None]]
# Note - this is bombing right now...

In [6]:
mfp = pyemu.helpers.PstFromFlopyModel(m,new_model_ws="schur_test",zone_props=zone_props,
                                          const_props=const_props,k_zone_dict=k_zone_dict,
                                          remove_existing=True,bc_props=bc_props)

2017-09-18 09:25:33.823006 starting: updating model attributes
2017-09-18 09:25:33.823207 finished: updating model attributes took: 0:00:00.000201
2017-09-18 09:25:33.823275 WARNING: removing existing 'new_model_ws

creating model workspace...
   schur_test

changing model workspace...
   schur_test
2017-09-18 09:25:33.949876 starting: writing new modflow input files
Util2d:delr: resetting 'how' to external
Util2d:delc: resetting 'how' to external
Util2d:model_top: resetting 'how' to external
Util2d:botm_layer_0: resetting 'how' to external
Util2d:botm_layer_1: resetting 'how' to external
Util2d:botm_layer_2: resetting 'how' to external
Util2d:ibound_layer_0: resetting 'how' to external
Util2d:ibound_layer_1: resetting 'how' to external
Util2d:ibound_layer_2: resetting 'how' to external
Util2d:strt_layer_0: resetting 'how' to external
Util2d:strt_layer_1: resetting 'how' to external
Util2d:strt_layer_2: resetting 'how' to external
Util2d:rech_1: resetting 'how' to external
Util2d:rech_

Util2d:rech_342: resetting 'how' to external
Util2d:rech_343: resetting 'how' to external
Util2d:rech_344: resetting 'how' to external
Util2d:rech_345: resetting 'how' to external
Util2d:rech_346: resetting 'how' to external
Util2d:rech_347: resetting 'how' to external
Util2d:rech_348: resetting 'how' to external
Util2d:rech_349: resetting 'how' to external
Util2d:rech_350: resetting 'how' to external
Util2d:rech_351: resetting 'how' to external
Util2d:rech_352: resetting 'how' to external
Util2d:rech_353: resetting 'how' to external
Util2d:rech_354: resetting 'how' to external
Util2d:rech_355: resetting 'how' to external
Util2d:rech_356: resetting 'how' to external
Util2d:rech_357: resetting 'how' to external
Util2d:rech_358: resetting 'how' to external
Util2d:rech_359: resetting 'how' to external
Util2d:rech_360: resetting 'how' to external
Util2d:rech_361: resetting 'how' to external
Util2d:rech_362: resetting 'how' to external
Util2d:rech_363: resetting 'how' to external
Util2d:rec

/Users/jwhite/anaconda/lib/python3.5/site-packages/pandas/core/indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/jwhite/anaconda/lib/python3.5/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


# Cool - let's take a quick look at the PST control file this made

In [14]:
inpst = mfp.pst

In [15]:
inpst.parameter_data

,parnme,partrans,parchglim,parval1,parlbnd,parubnd,pargp,scale,offset,dercom
welflux_000,welflux_000,log,factor,1.0,0.10,10.00,welflux,1.0,0.0,1
hk0_zn1.0,hk0_zn1.0,log,factor,1.0,0.01,100.00,_znhk0,1.0,0.0,1
hk0_zn2.0,hk0_zn2.0,log,factor,1.0,0.01,100.00,_znhk0,1.0,0.0,1
hk0_zn3.0,hk0_zn3.0,log,factor,1.0,0.01,100.00,_znhk0,1.0,0.0,1
hk0_zn4.0,hk0_zn4.0,log,factor,1.0,0.01,100.00,_znhk0,1.0,0.0,1
hk0_zn5.0,hk0_zn5.0,log,factor,1.0,0.01,100.00,_znhk0,1.0,0.0,1
hk0_zn6.0,hk0_zn6.0,log,factor,1.0,0.01,100.00,_znhk0,1.0,0.0,1
rech0_cn,rech0_cn,log,factor,1.0,0.75,1.25,_cn,1.0,0.0,1
sy0_cn,sy0_cn,log,factor,1.0,0.25,1.75,_cn,1.0,0.0,1


# Looks like we have multipliers on our zones. Let's set `noptmax` to -1 to calculate a Jacobian matrix and then see what `pyemu` functionality we can light up

## First, though, we can report all the control data to see `noptmax` and everything else

In [16]:
inpst.control_data.formatted_values

name
rstfle                        restart
pestmode                   estimation
npar                                9
nobs                            10598
npargp                              3
nprior                              0
nobsgp                             29
maxcompdim                          0
ntplfle                             4
ninsfle                             3
precis                         single
dpoint                          point
numcom                              1
jacfile                             0
messfile                            0
obsreref                   noobsreref
rlambda1                 2.000000E+01
rlamfac                 -3.000000E+00
phiratsuf                3.000000E-01
phiredlam                1.000000E-02
numlam                             -7
jacupdate                         999
lamforgive                 lamforgive
derforgive               noderforgive
relparmax                1.000000E+01
facparmax                1.000000E+01
facorig

In [17]:
inpst.control_data.noptmax=0
inpst.write(os.path.join('schur_test','freyberg_pest.pst'))

## If we just want to run in serial, we could run at the command line or we can use the following `pyemu` helper function to run `pestpp` for us. The cool thing is, this is operating system agnostic -- it will work regardless of platform.

# !!!
# _NB --> this is crashing because forward_run.py insists on the forward exe being MF2005 but there is a NWT file in the NAM file, so seems like it should be recognizing the model as MFNWT rather than MF2005._
# !!!

In [18]:
pyemu.helpers.run('pestpp freyberg_pest.pst', cwd='schur_test')

run():pestpp freyberg_pest.pst


In [12]:
inpst.model_command

['python forward_run.py']

In [13]:
m.exe_name

'mfnwt'